In [1]:
import environ
env = environ.Env()
environ.Env.read_env('.env')

In [2]:
KEY=env('OPEN_API_KEY')

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
from langchain_community.chat_models import ChatOllama
import PyPDF2

In [5]:
# Set up the OpenAI API client
#llm=ChatOpenAI(openai_api_key=KEY, temperature=0, model_name="gpt-3.5-turbo")

# Set up the Ollama
llama=ChatOllama(model=phi3)

NameError: name 'phi3' is not defined

In [12]:
RESPONSE_JSON = {
    "1": {
        "mcq": "soal pilihan ganda",
        "options": {
            "a": "pilihannya disini",
            "b": "pilihannya disini",
            "c": "pilihannya disini",
            "d": "pilihannya disini",
        },
        "correct": "correct answer",
    },
    "2": {
          "mcq": "soal pilihan ganda",
        "options": {
            "a": "pilihannya disini",
            "b": "pilihannya disini",
            "c": "pilihannya disini",
            "d": "pilihannya disini",
        },
        "correct": "correct answer",
    },
    "3": {
          "mcq": "soal pilihan ganda",
        "options": {
            "a": "pilihannya disini",
            "b": "pilihannya disini",
            "c": "pilihannya disini",
            "d": "pilihannya disini",
        },
        "correct": "correct answer",
    },
}

In [13]:
TEMPLATE="""
Text={text}
Nama kamu adalah DhankMaker, Kamu adalah seorang instruktur atau guru yang sangat handal dalam segala bidang, 
yang ditugaskan untuk membuat soal dan jawaban pilihan ganda menggunakan bahasa indonesia.
buat soal pilihan ganda yang jumlah soalnya {number} dengan tema {subject} dan diperuntukkan bagi siswa {level} dengan tingkat kesulitan {tone} .
pastikan bahwa soalnya tidak berulang dan pastikan semua soalnya dalam bentuk teks
pastikan format respon nya seperti RESPONSE_JSON dibawah dan gunakan itu sebagai petunjuk 
pastikan untuk membuat soal {number}
### RESPONSE_JSON
{response_json}
"""

In [14]:
quiz_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json", "level"],
    template=TEMPLATE,
)

In [29]:
quiz_chain=LLMChain(llm=llama, prompt=quiz_prompt, output_key="quiz", verbose=True)

In [16]:
TEMPLATE2="""
Kamu adalah seorang ahli tata bahasa indonesia dan penulis bahasa indonesia. Diberikan Kuis Pilihan Ganda untuk siswa {level}.
Anda perlu mengevaluasi kompleksitas pertanyaan dan memberikan analisis kuis secara lengkap. Hanya gunakan maksimal 50 kata untuk analisis kompleksitas. 
jika kuis tidak sesuai dengan kemampuan kognitif dan analitis siswa,
perbarui soal kuis yang perlu diubah dan ubah nadanya agar sesuai dengan kemampuan siswa
Kuis_MCQ:
{quiz}

Periksa dari Penulis ahli Bahasa Indoesia dari kuis di atas:
"""

In [30]:
quiz_eval=PromptTemplate(
    input_variables=["quiz","level"],
    template=TEMPLATE2
)

In [31]:
review_chain=LLMChain(llm=llama, prompt=quiz_eval, output_key="eval", verbose=True)

In [32]:
generate_eval=SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json", "level"],
    output_variables=["quiz", "eval"],
    verbose=True
)

In [33]:
with get_openai_callback() as cb:
    response=generate_eval({
        "text": "Soal Pilihan Ganda",
        "number": 10,
        "subject": "Fisika",
        "tone": "Mudah",
        "response_json": RESPONSE_JSON,
        "level": "Siswa SMP"
    })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text=Soal Pilihan Ganda
Nama kamu adalah DhankMaker, Kamu adalah seorang instruktur atau guru yang sangat handal dalam segala bidang, 
yang ditugaskan untuk membuat soal dan jawaban pilihan ganda menggunakan bahasa indonesia.
buat soal pilihan ganda yang jumlah soalnya 10 dengan tema Fisika dan diperuntukkan bagi siswa Siswa SMP dengan tingkat kesulitan Mudah .
pastikan bahwa soalnya tidak berulang dan pastikan semua soalnya dalam bentuk teks
pastikan format respon nya seperti RESPONSE_JSON dibawah dan gunakan itu sebagai petunjuk 
pastikan untuk membuat soal 10
### RESPONSE_JSON
{'1': {'mcq': 'soal pilihan ganda', 'options': {'a': 'pilihannya disini', 'b': 'pilihannya disini', 'c': 'pilihannya disini', 'd': 'pilihannya disini'}, 'correct': 'correct answer'}, '2': {'mcq': 'soal pilihan ganda', 'options': {'a': 'pilihannya disini', 'b': 'pilihannya disini', 'c': 'pilihannya disini', 'd

PermissionDeniedError: Error code: 403 - {'error': {'message': 'You are not allowed to sample from this model', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [27]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0
